# Solving Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Libraries and API Configuration

In [ ]:
# Install the OpenAI Python package.
%pip install openai

In [ ]:
%pip install langchain langchain_community langchain_chroma transformers sentence-transformers

In [5]:
import os

#TODO: Insert your OpenAI API key here
os.environ["OPENAI_API_KEY"] = 'Insert your API key here'

In [6]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"]
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [1]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

#TODO: Insert your document ID here
document_id = 'Insert your document ID here'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


# Task 1. Classify emails

We use prompt engineering to classify the messages.We create a new column in the dataframe with the results and save them as a CSV.

In [7]:
# Function to classify email using GPT-4
def classify_email(email_subject, email_body):
    # Prepare the prompt for GPT-4
    prompt = f"""
        You are an assistant helping a fashion store by classifying emails into two categories: "product inquiry" and "order request." Below is the subject and body of an email. Your job is to classify it into one of these categories:

        Email:
        ###
        Subject: {email_subject}
        Body: {email_body}
        ###

        Based on the content above, classify this email as either "product inquiry" or "order request". Do not write any additional text.

        Example:
        ###
        Subject: Need your help
        Body: I am interested in learning more about your products. Can you provide more information?
        ###

        Response: product inquiry

        """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=50
        )
    
    # Extract category from the response
    result = response.choices[0].message.content.strip().lower()
    
    # Return classification result
    if "inquiry" in result:
        return "product inquiry"
    elif "order" in result:
        return "order request"
    else:
        return "unknown"


In [ ]:
# Classify all emails and store results
emails_df['category'] = emails_df.apply(lambda row: classify_email(row['subject'], row['message']), axis=1)

# Output the results to the 'email-classification' sheet
classification_df = emails_df[['email_id', 'category']]
classification_df.to_csv('email-classification.csv', index=False)  # Saving temporarily as CSV before uploading

In [8]:
emails_df['category'].value_counts()

category
product inquiry    11
order request      10
Name: count, dtype: int64

# Task 2. Process order requests

First we create a new column with the product summary. This is what we will use to create our vector database to later query. Then, we use the LLM and prompt engineering to check for products and quantities on each email. Finally, we check the product and stock using RAG and process the order requests.

In [ ]:
# Create a new column that summarizes the product information
products_df['summary'] = products_df.apply(lambda row: f"product_id: {row['name']} - name:{row['name']} - category: {row['category']} - description: {row['description']} - stock: {row['stock']} - seasons: {row['seasons']} - price: {row['price']}", axis=1)
products_df.head()

In [ ]:
# Import Langchain and the necessary modules
from langchain_community.document_loaders import DataFrameLoader
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Load the product catalog from the DataFrame and store in a Chroma vector store
loader = DataFrameLoader(products_df, page_content_column="summary")
documents = loader.load()

huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs={'device':'cpu'}, 
    encode_kwargs={'normalize_embeddings': True}
)

# Create embeddings for products and initialize Chroma vector store
vectorstore = Chroma.from_documents(documents, embedding=huggingface_embeddings)

In [42]:
# Function to extract products and quantities from email body
def extract_products_and_quantities(email_body):
    #This needs to be improved to account for categories of products and seasons
    prompt = f"""
    You are a data extraction assistant. I will provide you with an email body that contains an order request. Your task is to extract the list of products and their respective quantities mentioned in the email.

    Here is the email body:
    ###
    "{email_body}"
    ###

    Please provide the extracted list of products and their quantities in the following format. Only include products that have a valid quantity specified. Only use numbers as a quantity.
    - Product Name: Quantity

    Example:
    - Blue Dresses: 2
    - Red Heels: 1
    """
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=50
        )
    
    extracted_text = response.choices[0].message.content.strip()
    # Convert extracted text to a dictionary
    product_list = {}
    for line in extracted_text.split('\n'):
        #print(line)
        if ': ' in line:
            product, quantity = line.split(': ')
            if quantity.isdigit():
                product_list[product.strip()] = int(quantity.strip())
            else:
                print(f"Quantity for product '{product}' is not a valid number. Data: {line}")
    
    return product_list

{'- CLF2109 Cable Knit Beanies': 5, '- FZZ1098 Fuzzy Slippers': 2}


In [45]:
# Function to process orders
def process_order_request(email_id, email_body):
    # Extract product and quantity from email body
    products_in_email = extract_products_and_quantities(email_body)
    order_status_list = []
    
    # Process each product in the order request
    for product_name, quantity in products_in_email.items():
        relevant_products = vectorstore.similarity_search(product_name)
        relevant_product = relevant_products[0].metadata['product_id']
        print(relevant_product)
        product_row = products_df[products_df['product_id'] == relevant_product]
        
        if not product_row.empty:
            product_id = product_row.iloc[0]['product_id']
            stock = product_row.iloc[0]['stock']
            #print(product_id, stock)
            #print("---")
            
            if stock >= quantity:
                # Order can be fulfilled
                products_df.loc[products_df['product_id'] == relevant_product, 'stock'] -= quantity
                order_status_list.append({
                    'email ID': email_id,
                    'product ID': product_id,
                    'quantity': quantity,
                    'status': 'created'
                })
            else:
                # Out of stock
                order_status_list.append({
                    'email ID': email_id,
                    'product ID': product_id,
                    'quantity': quantity,
                    'status': 'out of stock'
                })
        else:
            # Handle case where product is not found in catalog
            order_status_list.append({
                'email ID': email_id,
                'product ID': 'unknown',
                'quantity': quantity,
                'status': 'product not found'
            })
    
    return order_status_list

# Apply the process to order requests and generate the output for 'order-status'
order_requests = emails_df[emails_df['category'] == 'order request']
order_status_results = []

for _, row in order_requests.iterrows():
    order_status_results.extend(process_order_request(row['email_id'], row['message']))

# Convert the result to a DataFrame and save
order_status_df = pd.DataFrame(order_status_results)
order_status_df.to_csv('order-status.csv', index=False)  # Saving temporarily as CSV before uploading

Quantity for product '- SFT1098 Infinity Scarves' is not a valid number. Data: - SFT1098 Infinity Scarves: 3 to 4
CLF2109
CLF2109 2
---
FZZ1098
FZZ1098 2
---
VSC6789
VSC6789 6
---
PTR9876
PTR9876 1
---
SWL2345
SWL2345 5
---
RSG8901
RSG8901 1
---


# Task 3. Handle product inquiry

We use the vector database to retreive relevant data and serve it to the user based on the similarity to their email message.

In [32]:
# Function to handle product inquiries
def handle_product_inquiry(email_id, email_subject, email_body):
    relevant_products = vectorstore.similarity_search(str(email_subject) + " " + str(email_body))
    
    if relevant_products:
        response = f"Dear Customer, \n\nHere are the details for the products you inquired about:\n\nProduct Name: {relevant_products[0].metadata['name']}\nDescription: {relevant_products[0].metadata['description']}\nSeasons: {relevant_products[0].metadata['seasons']}\nPricing: ${str(relevant_products[0].metadata['price'])}\n\n"
        return response
    else:
        return "Dear Customer, \n\nWe could not find any relevant products based on your inquiry."

# Apply the function to generate responses for product inquiries
product_inquiries = emails_df[emails_df['category'] == 'product inquiry']
inquiry_responses = []

for _, row in product_inquiries.iterrows():
    response = handle_product_inquiry(row['email_id'], row['subject'], row['message'])
    inquiry_responses.append({'email ID': row['email_id'], 'response': response})

# Save the responses to 'inquiry-response' sheet
inquiry_response_df = pd.DataFrame(inquiry_responses)
inquiry_response_df.to_csv('inquiry-response.csv', index=False)  # Saving temporarily as CSV before uploading